# SMD Python Hands On

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

## Table of Contents
<div id="toc"></div>

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width: 95% !important; }</style>"))

## Basics

Not really the topic of today, these are good starting materials:

* [PeP et al. Toolbox Workshop (German)](https://toolbox.pep-dortmund.org) 
* [Scientific Python Notebooks](https://github.com/maxnoe/scientific_python_notebooks)
* [A Byte Of Python](https://python.swaroopch.com/)

Especially for the Data Mining part, but a generally a very good resource:
* [The Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)



## Style Guide and Linter

* For python code, there is a generally accepted style guide: pep8. Follow it!

* The `ruff` tool checks for both the syntactical errors and style guide violations
    * `mamba install ruff`
    * `flake8 test.py`
    
* Autoformatter
    * `ruff format` reformats files so they follow the styleguide → not have to think about it anymore
    
* All good editors support linter, autocompletion, etc., recommendation would be VS Code (or VS Codium):
  * VS Code: https://code.visualstudio.com/

## conda environments

* Different projects might need different versions of python and libraries
* Best practice: isolated environemnt with fixed versions
* Using conda: conda environments
* Conda environments can be specified in text files listing needed packages and their versions 


* conda can be slow when installing or updating packages. Mamba is a much better and faster alternative:
    * For a fresh installation (recommended!), download and install `miniforge3` here:  
      https://github.com/conda-forge/miniforge#miniforge3
    * For an existing conda installation, you can try:
      ```
      conda install -n base -c conda-forge mamba
      ``` 

Create a new environment using the definition file:

```
$ mamba env create -f environment.yml
```

To use the environment:
```
$ conda activate smd
```

Your python / ipython / jupyter should now come from this environment:
```
$ which python
/home/maxnoe/.local/conda/envs/smd/bin/python
```

## numpy

Short reminder of the most important numpy features for SMD

* Python is a dynamic, interpreted language. 
    * Easy to use
    * Powerful
    * Slow (large overhead, especially for simple numerical operations)
    
* Numpy is a library providing an efficient array implementation and access to fast code through compiled C++/C/Cython/Fortran code  
   ⇒ Application to large arrays as a whole or element-wise (vectorization)  
   ⇒ Rule of thumb: no for-loops over numpy arrays
   
* Many features for data analysis, random numbers, linear algebra, ...

* "Naive" python code being slow is one of the main criticisms of the language, especially for its use in science.

* Despite that, it is now the most commonly used language for data analysis.

* Most "hot" code is not actually python, but compiled code like numpy. Python is "the glue".

* There are several ways of making python code faster (e.g. numba, cython, ...) or replace it completely (Julia, C++, ...)

In [ ]:
import numpy as np

## Data Types

Numpy supports many different data types, most important are probably these three

* bool: True / False
* int64: 64-Bit signed integer
* float64: 64-Bit floating point number

More on data types and how they work in the next lecture, "Numerical Foundations"

In [ ]:
# numpy arrays from python lists
floats = np.array([1.0, 3.14, 1e3])
ints = np.array([1, 2, 3])
bools = np.array([True, False, True])

print(floats.dtype, ints.dtype, bools.dtype)

### Basic properties of numpy arrays

In [ ]:
array2d = np.array([
    [1, 2, 3],
    [4, 5, 6]
])

def array_info(a):
    print(f'{len(a)=}, {a.size=}, {a.ndim=}, {a.shape=}, {a.dtype=!s}')

array_info(floats)
array_info(array2d)

### Indexing & Masks

Numpy arrays can be indexed using (collections of) integers, slices or boolean masks

In [ ]:
a = np.array([1.0, -3.5, 42, -5])
a

In [ ]:
a[0], a[-1], a[1:-1], a[::2]

In [ ]:
a > 0

In [ ]:
a[a > 0]

In [ ]:
# parentheses are needed here
# | = or
# & = and
# ~ = not

a[(a > -10) & (a < 10)]

### The axis keyword

Important for aggregating operations (e.g. `np.sum`, `np.mean`, `np.prod`, `np.std`)

In [ ]:
X = np.arange(12).reshape(4, 3)
X

In [ ]:
np.sum(X)

In [ ]:
np.sum(X, axis=0)

### Broadcasting

Docs: https://numpy.org/doc/stable/user/basics.broadcasting.html

Using "broadcasting", numpy can apply element-wise calculations to arrays of different sizes, if the shapes are *compatible*

* The last, overlapping dimensions must be *compatible*

* Compatible means

    * both dimensions are the same
    * or one dimension is 1


Examples:
* shape `(3, 2, 2)` is compatible with shape `(2, 2)`, the last dimensions are the same 
* shape `(3, 2, 2)` is compatible with shape `(1, 2)`, since the last dimensions are either the same, or one of them is 1
* shape `(3, 2, 2)` is not compatible with shape `(3, 2)`, as last dimensions are note the same (the first ones are though) 
* shape `(3, 2, 2)` is compatible with `(3, 2, 1)` since all dimensions are equal or 1

A a new dimension of size `1` can easily be inserted into any array using `np.newaxis`, making shapes compatible, see below

In [ ]:
a = np.arange(12).reshape(4, 3)
b = 5
c = np.arange(3)
d = np.arange(4)

In [ ]:
a.shape, c.shape, d.shape

In [ ]:
a - b

In [ ]:
a - c

In [ ]:
# this error is expected
a - d

we can add a new dimension of size 1 at the and to make the shapes compatible:

In [ ]:
a.shape, d.shape, d[..., np.newaxis].shape

In [ ]:
a - d[:, np.newaxis]

### Performance example: Loops vs. Numpy vs. Numba

Task: Find closest point to a reference point in a list of points

In [ ]:
point = [0, 1]

points = [
    (0, 0),
    (0.5, -0.5),
    (1, -1),
    (0, 2),
    (0, 1.1),
    (-2, 3),
    (5, 1),
    (10, 4),
    (-4, 2),
    (-3, 0),
] * 1000

#### Pure python using a for-loop over a list of tuples and the math module:

In [ ]:
import math


def distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)


def find_closest(points, point):
    min_distance = math.inf
    min_idx = None
    
    for i, other in enumerate(points):
        d = distance(point, other) 
        
        if d < min_distance:
            min_distance = d
            min_idx = i
    
    return min_idx

In [ ]:
idx = find_closest(points, point)
print(idx, points[idx])

In [ ]:
%%timeit -o 
find_closest(points, point)

In [ ]:
timeit_python = _

#### Using numpy arrays and numpy methods

In [ ]:
points = np.array(points)
point = np.array(point)

In [ ]:
def find_closest_numpy(points, point):
    distances = np.linalg.norm(points - point, axis=1)
    idx = np.argmin(distances)
    return idx


idx = find_closest_numpy(points, point)
print(idx, points[idx])

In [ ]:
%%timeit -o
find_closest_numpy(points, point)

In [ ]:
timeit_numpy = _ 

And now using numba

In [ ]:
from numba import njit

# a decorator is a function that changes a function
# we apply the njit decorator to our python functions:
distance = njit(distance)
find_closest = njit(find_closest)



# normally used like this:
@njit
def foo():
    return 1

In [ ]:
idx = find_closest(points, point)

print(idx, points[idx])

In [ ]:
%%timeit -o
find_closest(points, point)

In [ ]:
timeit_numba = _

In [ ]:
factor = timeit_python.average / timeit_numpy.average
print(f'Numpy is {factor:.1f} times faster than python')

In [ ]:
factor = timeit_numpy.average / timeit_numba.average
print(f'Numba is {factor:.1f} times faster than numpy')

In [ ]:
factor = timeit_python.average / timeit_numba.average
print(f'Numba is {factor:.1f} times faster than python')

### What kind of sorcery is this?

* Python is a very *dynamic*, interpreted language. That makes it very powerful, but also has a large overhead, especially for simple numerical calculations

* Numpy is a library providing generic methods in compiled, optimized Cython, C, Fortran, ... which has much less overhead. But since every operation works on the whole array, intermediate arrays are created for the intermediate results

* Numba takes the python code and "just-in-time (Jit)" compiles it to machine code, completely eliminating the need for intermediate arrays and thus memory allocations in this case.


The speed-ups can be immense and numba compiled python code can be as-fast or even faster than hand-optimized C / C++ code and has trivial interoperability with python and numpy.


For SMD, numpy will almost always be "good enough", numba won't be needed to make exercises run in less than a couple of seconds to minutes.

But: numba is very well suited in general and especially for algorithms which cannot be vectorized, e.g. where the results depend on the previous iterations

### Pseudo Random Numbers

Docs: https://numpy.org/doc/stable/reference/random/index.html

In [ ]:
from numpy.random import default_rng

# create a new random generator with a fixed seed
# this avoids "evil" global state when using `np.random.seed` or `np.random.<function>`
rng = default_rng(42)

#### 1-D Distributions

In [ ]:
uniform = rng.uniform(-5, 5, 1000)
normal = rng.normal(0, 1, 1000)
poisson = rng.poisson(3, 1000)

#### N-D Normal Distribution

In [ ]:
mean = [2, 1]
cov = [[2, 1],
       [1, 4]]

normal_2d = rng.multivariate_normal(mean, cov, 1000)

Important for reproducibility: setting the seed.

Also important for parallel calculations or for resuming simulations.

In [ ]:
rng.normal()

In [ ]:
rng = default_rng(0)
rng.normal()

## matplotlib

Short intro into the two most important types of plots for SMD:

* Histograms
* Scatter plots

### Histograms

Histograms count the number of samples in specific intervals

In [ ]:
import matplotlib.pyplot as plt

# for interactive plots in the notebook (using the ipympl package).
# in ipython just use %matplotlib
%matplotlib widget

In [ ]:
# relatively new feature of matplotlib, use this instead of `fig.tight_layout()`
fig, ax = plt.subplots(constrained_layout=True)

ax.hist(normal, bins=20, range=[-5, 5])

None # hides matplotlib objects in output, just to not mess up the notebook

When comparing different samples, it's important to use the same "binning"

In [ ]:
bins = 20
limits = [-5, 5]

combined = np.concatenate([normal, uniform])

fig, ax = plt.subplots(constrained_layout=True)

# define common options in a single location
hist_options = dict(
    bins=bins,
    range=limits,
    histtype='step',
    lw=2,
)

ax.hist(normal, label='Normal', **hist_options)
ax.hist(uniform, label='Uniform', **hist_options)
ax.hist(combined, label='Combined', linestyle=':',color='k', **hist_options)

ax.legend()

None  # just to not mess up the notebook

<span style="color: crimson; font-weight: bold; font-size: 2rem">
    For discrete values (integers), always use integral-width bins centered around the values
</span>

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.hist(poisson, bins=15)
None

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax.hist(
    poisson,
    bins=np.arange(15) - 0.5,  # bins can be either number of bins or bin edges
    edgecolor='w',
    lw=2
)


np.arange(15) - 0.5

In [ ]:
# convert bin edges to bin centers and bin widths
bins = np.arange(8) - 0.5

bin_centers = 0.5 * (bins[:-1] + bins[1:])
bin_widths = np.diff(bins)

print(bins)
print(bin_centers)
print(bin_widths)

### Scatter Plots

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax.scatter(normal_2d[:, 0], normal_2d[:, 1])

None

In [ ]:
normal_2d = rng.multivariate_normal(mean, cov, 10000)

fig, ax = plt.subplots(constrained_layout=True)

# smaller dots, no border and some transparency for this many points
ax.scatter(
    normal_2d[:, 0],
    normal_2d[:, 1],
    s=5,
    alpha=0.2,
    linewidth=0,
)

None

A more advanced example:

* Points can be colored using a third array
* Using ListedColorMap, you can get a discrete colormap for each class

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

In [ ]:
from matplotlib.colors import ListedColormap

n_classes = len(iris.target_names)

# new ColorMap with `n_classes` discrete colors 
# using the standard color rotation C0, C1, ...
cmap = ListedColormap([f'C{i}' for i in range(n_classes)], name='iris')
cmap

In [ ]:
# automatically adjust spacing
fig, ax = plt.subplots(constrained_layout=True)

# scatter plot with colors per class
scat = ax.scatter(
    iris.data[:, 0],       # x-values, first column
    iris.data[:, 1],       # y-values, second column 
    c=iris.target,         # data to determine color
    cmap=cmap,             # colormap, converts data in c to an actual color
    vmin=-0.5,             # minimum value for the color axis
    vmax=n_classes - 0.5,  # maximum value for the color axis
)


# follow SI conventions (divide by unit)
ax.set_xlabel(iris.feature_names[0].replace('(cm)', '/ cm'))
ax.set_ylabel(iris.feature_names[1].replace('(cm)', '/ cm'))

# colorbar with ticklabels
bar = fig.colorbar(scat, ticks=[0, 1, 2], ax=ax)
bar.set_ticklabels(iris.target_names)

### 2D-Histograms

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

hist, xedges, yedges, plot = ax.hist2d(
    normal_2d[:, 0],
    normal_2d[:, 1],
    bins=[21, 21],
    range=[[-3, 7], [-7, 9]],
    cmap='inferno', # has more contrast than the default viridis
)
fig.colorbar(plot, ax=ax)
None

Sometimes helpful: Logarithmic scale for the color map

In [ ]:
# two different distributions, one significantly smaller
normal_2d = rng.multivariate_normal(mean, cov, 100000)
normal_2d_2 = rng.multivariate_normal([-1.5, 4.5], [[0.5, 0], [0, 0.5]], 500)

normal_2d_both = np.concatenate([normal_2d, normal_2d_2], axis=0)
normal_2d_both.shape

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

*_, plot = ax.hist2d(
    normal_2d_both[:, 0],
    normal_2d_both[:, 1],
    bins=[50, 50],
    range=[[-3, 7], [-7, 9]],
    cmap='inferno'
)
fig.colorbar(plot, ax=ax)

None

In [ ]:
from matplotlib.colors import LogNorm


fig, ax = plt.subplots(constrained_layout=True)

*_, plot = ax.hist2d(
    normal_2d_both[:, 0],
    normal_2d_both[:, 1],
    bins=[50, 50],
    range=[[-3, 7], [-7, 9]],
    norm=LogNorm(),
    cmap='inferno'
)
fig.colorbar(plot, ax=ax)

None

Also often important: fixed aspect ratio of the x and y axis:

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

hist, xbins, ybins, plot = ax.hist2d(
    normal_2d_both[:, 0],
    normal_2d_both[:, 1],
    bins=[50, 50],
    range=[[-3, 7], [-7, 9]],
    norm=LogNorm(),
    cmap='inferno',
)

fig.colorbar(plot, ax=ax)

# force x and y to have same aspect ratio, also takes numbers
ax.set_aspect('equal')


None

## scipy

For SMD-A, `scipy.stats` will be most important

### scipy.stats

Many statistical distributions with many properties

Docs: https://docs.scipy.org/doc/scipy/reference/stats.html

In [ ]:
from scipy.stats import norm

mean = 5
std = 2

normal_distribution = norm(mean, std)

In [ ]:
# draw random samples using our generator
samples = normal_distribution.rvs(size=1000, random_state=rng)

x = np.linspace(mean - 5 * std, mean + 5 * std, 250)

fig, ax = plt.subplots(constrained_layout=True)

# plot histogram
ax.hist(
    samples,
    bins=100,
    range=[x.min(), x.max()],
    density=True,  # normalize histgram to an area of 1
    label='Normalized Histogram',
)

# plot pdf and cdf
ax.plot(x, normal_distribution.pdf(x), label='Probability Density Function', lw=2)
ax.plot(x, normal_distribution.cdf(x), label='Cumulative Distribution Function', lw=2)


ax.legend()

None

In [ ]:
x = rng.normal(5, 2, 100)

# maximum likelihood fit 
mean_fit, std_fit = norm.fit(x)

mean_fit, mean, std_fit, std

## Pandas

Library for tabular data

In [ ]:
import pandas as pd

### Create a pandas.DataFrame from numpy arrays

In [ ]:
signal = pd.DataFrame({
    'x': rng.normal(2, 0.5, 1000),
    'y': rng.uniform(0.5, 1, 1000),
    'N': rng.poisson(50, 1000),
    't': rng.exponential(5, 1000),
})

### First look at the data

In [ ]:
signal.head()

### Input/Output

In [ ]:
signal.to_csv('data.csv')

In [ ]:
signal = pd.read_csv('data.csv', index_col=0)

In [ ]:
# HDF5 is a fast, binary data format better suited for large datasets
signal.to_hdf('data.hdf5', key='signal')

In [ ]:
background = pd.DataFrame({
    'x': rng.uniform(-4, 4, 10000),
    'y': rng.uniform(-4, 4, 10000),
    'N': rng.poisson(30, 10000),
    't': rng.exponential(10, 10000),
})

In [ ]:
# you can store more than one dataset in the same file
background.to_hdf('data.hdf5', key='background')

In [ ]:
background.head()

In [ ]:
df = pd.read_hdf('data.hdf5', key='signal')

In [ ]:
len(df)

Look at the first / last values

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

### The titanic dataset

In [ ]:
df = pd.read_csv('titanic.csv')

In [ ]:
df.head()

How many valid values in each column?

In [ ]:
df.count().sort_values(ascending=False)

Drop columns with too many missing values:

In [ ]:
# axis=1: drop columns
# inplace=True: directly change df
df.drop(['cabin', 'boat', 'body', 'home.dest'], axis="columns", inplace=True) 

df.head()

How many men/women on the titanic?

In [ ]:
df['sex'].value_counts()

Very powerful operation: GroupBy → Aggregate

The dataset is split into multiple groups, aggregated values calculated per group.


Example: mean survival rate by sex:

In [ ]:
df.groupby('sex')['survived'].agg('mean')

`DataFrame` also supports mask indexing:

In [ ]:
df['child'] = df['age'] < 9

df[df['child']]

In [ ]:
df.groupby('child').survived.mean()

## Classes

The overaching task this year is a simplified detector simulation and analysis.

Much of the structure will be predefined, with exercises "filling in blanks".


To understand the structure, some understanding of python classes and tests is helpful

Docs: https://docs.python.org/3/reference/datamodel.html

This is a **very** short primer

In [ ]:
class Detector:
    
    # special method to initialize a new instance of a class
    def __init__(self, width, height):
        self.width = width
        self.height = height

In [ ]:
detector = Detector(width=10, height=5)

print(detector.width, detector.height)

One possible exercise could be to implement a method checking if a particle hits the detector.

Let the coordinate system be defined as follows:
* Lower left corner of the detector is at (0, 0)
* width along x
* height along y


The given structure could then be:

In [ ]:
class Detector:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    def is_inside(self, x, y):
        # this is to make the function work without error,
        # but it does not give the correct answer yet
        inside = False
        ###### Implement your solution between here #######
        
        ###### and here #######
        return inside

We will provide tests for most of these, where you can check if your implementation
is correct. For this, we use pytest:

* Docs: https://docs.pytest.org/en/stable/
* Lecture on testing with pytest from the Escape School 2021: https://www.youtube.com/watch?v=pGg97d8TQuY

In [ ]:
def test_is_inside():
    # a test function to be used with `pytest`
    d = Detector(5, 5)

    assert d.is_inside(2, 2), 'This point should be inside'
    assert not d.is_inside(10, 10), 'This point should not be inside'
    print("test passed")
    
test_is_inside()

In Python, we can implement "special" operations on our class using *magic* methods,
which are defined in the data model and for example make operator overloading possible.

These methods always start and end with `__` (double-underscore, "dunder")

In [ ]:
print(Detector(10, 5))

In [ ]:
class Detector:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def __repr__(self):
        '''This methods enables conversion to a representation str, e.g. for printing'''
        return f'{self.__class__.__name__}(width={self.width}, height={self.height})'

In [ ]:
d = Detector(10, 5)
print(d)